<a href="https://colab.research.google.com/github/antfolk/BMEN35/blob/main/Session5/BMEN35_Ex15_deep_neural_networks_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Networks part 3
 
In this notebook we will take a look "under the hood" of one our trained models. 

We will ... you know the drill.

In [ ]:
# The usual imports
from numpy import argmax
from numpy import arange
from numpy import transpose
from numpy import array
from numpy import concatenate
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
import time
import numpy as np
from google.colab import files

We will start by uploading our models from before.

In [ ]:
_ = files.upload() # Select all the model files (*.h5)

Lets first load one of our models

In [ ]:
model = load_model('model_b.h5')

Lets remind ourselves of what the model looks like and investigate the convolutional layers.

In [ ]:

# investigate size of convolutional filters
for layer in model.layers:
  # check for convolutional layer
  if 'conv' not in layer.name:
      continue
  # get filter weights
  filters, biases = layer.get_weights()
  print(layer.name, filters.shape)

plot_model(model, show_shapes=True, show_layer_names=True, show_layer_activations=True) # 

Next, lets visualise the 32 filter in the convolutional layer. They are all of size 3x3, only one color/channel.

In [ ]:
# retrieve weights from the first hidden layer
filters, biases = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# plot first few filters
n_filters, ix = 1, 1
for i in range(32):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(1):
    # specify subplot and turn of axis
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
    # plot filter channel in grayscale
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
plt.show()

Next we will load some images and see what happens

In [ ]:
_ = files.upload() # Select sample_image1.png and sample_image2.png

In [ ]:
img1 = load_img('sample_image1.png', color_mode = "grayscale", target_size=(28, 28))
img2 = load_img('sample_image2.png', color_mode = "grayscale", target_size=(28, 28))
# plot the sample
plt.subplot(1, 2, 1)
plt.title('Image 1')
plt.imshow(img1, cmap='gray')
plt.subplot(1, 2, 2)
plt.title('Image 2')
plt.imshow(img2, cmap='gray')
plt.show()
# convert to array
img1 = img_to_array(img1)
img2 = img_to_array(img2)
# reshape into a single sample with 1 channel
img1 = img1.reshape(1, 28, 28, 1)
img2 = img2.reshape(1, 28, 28, 1)
# prepare pixel data
img1 = img1.astype('float32')
img1 = img1 / 255.0
img2 = img2.astype('float32')
img2 = img2 / 255.0

Lets start with using model c and see how this model performs. 

In [ ]:
model = load_model('model_c.h5')
predict_value1 = model.predict(img1)
digit1 = argmax(predict_value1)
predict_value2 = model.predict(img2)
digit2 = argmax(predict_value2)   
print('Predict image1 as: ', digit1)
print('Predict image2 as: ', digit2)

Lets plot the full model so we remember what it looks like.

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True, show_layer_activations=True) # 

Now we will have a look at the model at different layers. We will start with after the first hidden layers.

In [ ]:
# redefine model to output right after the first hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[0].output)
feature_maps = model.predict(img1)
print('Feature map size after first layer: ',feature_maps.shape)

In [ ]:
# plot all 64 maps in an 8x8 squares
ix = 1
for _ in range(8):
  for _ in range(4):
    # specify subplot and turn of axis
    ax = plt.subplot(8, 4, ix)
    ax.set_xticks([])
    ax.set_yticks([])
    # plot filter channel in grayscale
    plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
    ix += 1
# show the figure
plt.show()  

Now lets check the next layer. We will reload the model.


In [ ]:
# Reload model	
model = load_model('model_c.h5')
# redefine model to output right after the third hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[2].output)
feature_maps = model.predict(img1)
print(feature_maps.shape)
# plot all 64 maps in an 8x8 squares
ix = 1
for _ in range(8):
    for _ in range(4):
        # specify subplot and turn of axis
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
# show the figure
plt.show() 

And the layer after that.

In [ ]:
# Reload model	
model = load_model('model_c.h5')
# redefine model to output right after the fourth hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[3].output)
feature_maps = model.predict(img1)
print(feature_maps.shape)
# plot all 64 maps in an 8x8 squares
ix = 1
for _ in range(8):
    for _ in range(4):
        # specify subplot and turn of axis
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
# show the figure
plt.show()    

Now we will check the final layers.

In [ ]:
# Reload model	
model = load_model('model_c.h5')
# redefine model to output right after the first hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[6].output)
feature_maps6_1 = model.predict(img1)
feature_maps6_2 = model.predict(img2)   

# Reload model	
model = load_model('model_c.h5')
# redefine model to output right after the first hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[7].output)
feature_maps7_1 = model.predict(img1)
feature_maps7_2 = model.predict(img2)

# Plot the stuff
plt.figure(figsize=(10, 8)) 
plt.subplot(2, 2, 1)
plt.title('Output of layer 6 for image 1')
plt.plot(concatenate(feature_maps6_1), 'b-')
plt.subplot(2, 2, 2)
plt.title('Output of layer 6 for image 2')
plt.plot(concatenate(feature_maps6_2), 'r-')
plt.subplot(2, 2, 3)
plt.title('Output of layer 7 for image 1')
plt.plot(concatenate(feature_maps7_1), 'bx')
plt.subplot(2, 2, 4)
plt.title('Output of layer 7 for image 2')
plt.plot(concatenate(feature_maps7_2), 'rx')   
plt.show()  

What does the last row of plots represent?

# The End